In [205]:
import pandas as pd
import numpy as np
import functions_YoDa
#讀取courseDetail
courseDetail = pd.read_csv('data/course.csv')[['course code', 'Number of students', 'instructor']]
courseDetail['course code']=courseDetail['course code'].astype(str)
RoomDetail = pd.read_csv('data/classroom.csv')[['classroom', 'cr_capacity']]

#variables
roomNum=3 #教室數量
weekdays=5 #上課日子
dailyParts=3 #parts=將一天劃分為[早上、下午、晚上]
period=weekdays*dailyParts #15 一個weekdays中，不分教室的區塊總數
session=roomNum*weekdays*dailyParts #45 一個weekdays中，空教室的總數(一維陣列的長度)
k=weekdays*roomNum #15 [早上、下午、晚上] 一個part中的session數(索引調整參數)
totalCourseNum=30

# testing data
temp_schedule_simple=['306000001','','','','','','','','','','','','','','',
 '','307857001','','','','','','','','','356395001','','','','306737001',
'307932001','','356822001','','','','','','','','','','','','']
temp_schedule_feasible=['306000001','','306008001','','','306016002','356425001','','306016012','307873001','','307867001','307942001','','307870001',
 '306016022','307857001','306050011','356387001','356388001','307851001','306525001','','307035001','306736001','356395001','307034001','356461001','','306737001',
'307932001','','356822001','','356389001','356019001','356564001','','307901001','356813001','','356808001','','','']



In [203]:
from random import choice, randint, random
from string import ascii_lowercase
from copy import deepcopy

#https://github.com/100/Solid/edit/master/Solid/TabuSearch.py
from abc import ABCMeta, abstractmethod
from copy import deepcopy
from collections import deque
from numpy import argmax

class TabuSearch:
    """
    Conducts tabu search
    """
    __metaclass__ = ABCMeta

    cur_steps = None

    tabu_size = None
    tabu_list = None

    initial_state = None
    current = None
    best = None

    max_steps = None
    max_score = None

    def __init__(self, initial_state, tabu_size, max_steps, max_score=None):
        """

        :param initial_state: initial state, should implement __eq__ or __cmp__
        :param tabu_size: number of states to keep in tabu list
        :param max_steps: maximum number of steps to run algorithm for
        :param max_score: score to stop algorithm once reached
        """
        self.initial_state = initial_state

        if isinstance(tabu_size, int) and tabu_size > 0:
            self.tabu_size = tabu_size
        else:
            raise TypeError('Tabu size must be a positive integer')

        if isinstance(max_steps, int) and max_steps > 0:
            self.max_steps = max_steps
        else:
            raise TypeError('Maximum steps must be a positive integer')

        if max_score is not None:
            if isinstance(max_score, (int, float)):
                self.max_score = float(max_score)
            else:
                raise TypeError('Maximum score must be a numeric type')

    def __str__(self):
        return ('TABU SEARCH: \n' +
                'CURRENT STEPS: %d \n' +
                'BEST SCORE: %f \n' +
                'BEST MEMBER: %s \n\n') % \
               (self.cur_steps, self._score(self.best), str(self.best))

    def __repr__(self):
        return self.__str__()

    def _clear(self):
        """
        Resets the variables that are altered on a per-run basis of the algorithm

        :return: None
        """
        self.cur_steps = 0
        self.tabu_list = deque(maxlen=self.tabu_size)
        self.current = self.initial_state
        self.best = self.initial_state

    @abstractmethod
    def _score(self, state):
        """
        Returns objective function value of a state

        :param state: a state
        :return: objective function value of state
        """
        pass

    @abstractmethod
    def _neighborhood(self):
        """
        Returns list of all members of neighborhood of 
        state, given self.current

        :return: list of members of neighborhood
        """
        pass

    def _best(self, neighborhood):
        """
        Finds the best member of a neighborhood

        :param neighborhood: a neighborhood
        :return: best member of neighborhood
        """
        return neighborhood[argmax([self._score(x) for x in neighborhood])]

    def run(self, verbose=True):
        """
        Conducts tabu search

        :param verbose: indicates whether or not to print progress regularly
        :return: best state and objective function value of best state
        """
        self._clear()
        for i in range(self.max_steps):
            self.cur_steps += 1

            if ((i + 1) % 100 == 0) and verbose:
                print(self)

            neighborhood = self._neighborhood()
            neighborhood_best = self._best(neighborhood)

            while True:
                if all([x in self.tabu_list for x in neighborhood]):
                    print("TERMINATING - NO SUITABLE NEIGHBORS")
                    return self.best, self._score(self.best)
                if neighborhood_best in self.tabu_list:
                    if self._score(neighborhood_best) > self._score(self.best):
                        self.tabu_list.append(neighborhood_best)
                        self.best = deepcopy(neighborhood_best)
                        break
                    else:
                        neighborhood.remove(neighborhood_best)
                        neighborhood_best = self._best(neighborhood)
                else:
                    self.tabu_list.append(neighborhood_best)
                    self.current = neighborhood_best
                    if self._score(self.current) > self._score(self.best):
                        self.best = deepcopy(self.current)
                    break

            if self.max_score is not None and self._score(self.best) > self.max_score:
                print("TERMINATING - REACHED MAXIMUM SCORE")
                return self.best, self._score(self.best)
        print("TERMINATING - REACHED MAXIMUM STEPS")
        return self.best, self._score(self.best)


In [200]:
class Algorithm(TabuSearch):
    """
    Tries to get a randomly-generated string to match string "clout"
    """
    def _neighborhood(self):
        member = list(self.current)
        neighborhood = []
        for _ in range(10):
            neighbor = deepcopy(member)
            neighbor[randint(0, 44)] = choice(ascii_lowercase)
            neighbor = ''.join(neighbor)
            neighborhood.append(neighbor)
        return neighborhood

    def _score(self, state):
        return float(sum(state[i] == "123456789012345678901234567890123456789012345"[i] for i in range(45)))


def test_algorithm():
    algorithm = Algorithm('abcdefghijabcdefghijabcdefghijabcdefghijabcde', 50, 500, max_score=None)
    algorithm.run()

In [201]:
class Algorithm(TabuSearch):
    """
    Tries to get a randomly-generated string to match string "clout"
    """
    def _neighborhood(self):
        member = list(self.current)
        neighborhood = []
        for _ in range(10):
            neighbor = deepcopy(member)
            neighbor[randint(0, 9)] = choice(ascii_lowercase)
            #neighbor = ''.join(neighbor) #list to string
            neighborhood.append(neighbor)
        return neighborhood

    def _score(self, state):
        return float(sum(state[i] == ['a','a','a','a','a','a','a','a','a','a'][i] for i in range(10))) #填入obj. fum.


def test_algorithm():
    algorithm = Algorithm(['b','b','b','b','b','b','b','b','b','b'], 50, 500, max_score=None) #填入initial solution
    algorithm.run()

In [206]:
class Algorithm(TabuSearch):
    """
    Tries to get  
    """
    def _neighborhood(self):
        member = list(self.current)
        neighborhood = []
        for _ in range(10):
            neighbor = deepcopy(member)
            #SWAP:
            for _ in range(30):
                NeighborList = list(range(session))
                session1 = choice(NeighborList)
                NeighborList.remove((session1//roomNum)*roomNum+0)
                NeighborList.remove((session1//roomNum)*roomNum+1)
                NeighborList.remove((session1//roomNum)*roomNum+2)
                session2 = choice(NeighborList)
                SwapTemp = neighbor[session1] 
                neighbor[session1]  = neighbor[session2] 
                neighbor[session2]  = SwapTemp
            neighborhood.append(neighbor)
        return neighborhood

    def _score(self, state):
        return functions_YoDa.ObjFun(schedule,courseDetail, roomNum, k, RoomDetail, session, period, totalCourseNum) #填入obj. fum.

def test_algorithm():
    algorithm = Algorithm(temp_schedule_feasible, 50, 500, max_score=None) #填入initial solution
    algorithm.run() 

test_algorithm()

TypeError: ObjFun() takes 5 positional arguments but 8 were given

In [191]:
#test 
NerborList = list(range(session))
session1 = choice(NerborList)
NerborList.remove((session1//roomNum)*roomNum+0)
NerborList.remove((session1//roomNum)*roomNum+1)
NerborList.remove((session1//roomNum)*roomNum+2)
session2 = choice(NerborList)
SwapTemp = neighbor[session1] 
print(session1)
print(session2)

7
16


In [158]:
#test 
#neighborhood[argmax([functions_YoDa.ObjFun(temp_schedule_feasible,courseDetail, roomNum, k, RoomDetail) for x in neighborhood])]

for x in neighborhood : 
    pass
    #print('1')
    #print(x)
    
argmax([functions_YoDa.ObjFun(x,courseDetail, roomNum, k, RoomDetail) for x in neighborhood])


2

In [ ]:
#test 
temp_schedule_feasible
member = list(temp_schedule_feasible)
print(member)
neighborhood = []
for _ in range(3):
    neighbor = deepcopy(member)
    #SWAP:
    for _ in range(30):
        session1 = randint(0, session-1)
        session2 = randint(0, session-1)
        SwapTemp = neighbor[session1] 
        #print('SwapTemp  '+ SwapTemp)
        neighbor[session1]  = neighbor[session2] 
        #print( 'neighbor[session1]  '+ neighbor[session1])
        neighbor[session2]  = SwapTemp
        #print('neighbor[session2]  '+  neighbor[session2])
        #neighbor = ''.join(neighbor) #list to string
    neighborhood.append(neighbor)
neighborhood

In [44]:
#test 
import functions_YoDa
print( functions_YoDa.ObjFun(temp_schedule_feasible,courseDetail, roomNum, k, RoomDetail) )

import feasible_test as ft
ft.feasible_test(schedule)

3398.709315375982


True